In [1]:
import glob
import os
import re
import random
import pandas as pd
import re
import nltk
import string

In [2]:
default = ['title', 'abstract', 'introduction', 'related work', 'experiments', 'experiment', 'result', 
           'results', 'experimental-setup', 'experimental setup', 'hyperparameter', 'hyperparameters',
           'ablation analysis', 'conclusion', 'background', 'future work', 'discussion', 'method']

In [3]:
def read_integers(filename):
    with open(filename) as f:
        return [int(x) for x in f]

In [4]:
def compare(s1, s2):
    s1 = re.sub('[^A-Za-z0-9]+', ' ', s1).lower()
    s2 = re.sub('[^A-Za-z0-9]+', ' ', s2).lower()
    l1 = s1.split()
    l2 = s2.split()
    l2 = [token for token in l2 if not token.isnumeric()]
    l2 = [token for token in l2 if not token == 'I']
    return l1 == l2, l1

In [7]:
path = "../NCG_Dataset/training-data-master/*/*"
directory = glob.glob(path)
position = len(path)-3
df = pd.DataFrame(columns = ['topic', 'paper_ID', 'text', 'main_heading', 'sub_heading', 
                             'label', 'pos1', 'pos2', 'pos3', 'citation'])
ID_list, topic_list, text = [], [], []
pos1, pos2, pos3, citations = [], [], [], []
main_heading, sub_heading, labels = [], [], []
length, count = -1, 0
for dir_path in directory:
    
    # Directory of the paper
    #print(dir_path)
    
    file = "".join((dir_path,"/*-Stanza-out.txt"))
    stanza_file = glob.glob(file)[0]
    grobid_file = stanza_file.replace("Stanza", "Grobid")
    sent_file = "".join((dir_path,"/sentences.txt"))
    main_file = "".join((dir_path,"/main_heading.txt"))
    sub_file = "".join((dir_path,"/sub_heading.txt"))
    
    # Reading sentences from stanza file
    with open(stanza_file, encoding='ISO-8859-1') as f:
        lines = f.readlines()
    lines = [line.strip() for line in lines]
    
    # Extracting Topic and Paper_ID of the current paper
    pos = dir_path.find('/', position)
    topic = dir_path[position:pos]
    ID = dir_path[pos+1:len(dir_path)]
    
    # Reading Integers (sentence number) from sentence.txt
    sent = read_integers(sent_file)
    
    # Check if main heading file exists
    if (os.path.isfile(main_file)):
        # Reading Main Heading from main_heading.txt
        with open(main_file, encoding='ISO-8859-1') as f:
            headings = f.readlines()
        # Reading Sub Heading from sub_heading.txt
        with open(sub_file, encoding='ISO-8859-1') as f:
            sub_headings = f.readlines()
        # If heading file conain only title and abstract
        if(len(headings) <= 4):
            headings = sub_headings = default
            print(dir_path)
            count += 1
    else:
        headings = sub_headings = default
        print(dir_path)
        count += 1
        
    #Remove end of line from each line
    headings = [head.strip() for head in headings]
    sub_headings = [head.strip() for head in sub_headings]
    
    #Add paper_id and topic to list
    ID_list.extend([ID]*len(lines))
    topic_list.extend([topic]*len(lines))
    
    main_head, sub_head = "", ""
    # Enumerating lines in stanza file
    for i, line in enumerate(lines):
        
        # Check whether sentence contain a citation
        citation = 1 if re.findall(r'\(([^)]+)?(?:19|20)\d{2}?([^)]+)?\)', line) else 0
        citation = 1 if re.findall(r'et al', line) else citation
        
        # Check whether sentence is a contribution sentence
        label = 1 if (i+1 in sent) else 0
        
        head_flag, sub_flag = False, False
        
        # Check whether this sentence is a heading
        for head in headings:
            flag, temp_list = compare(head, line)
            if(flag and len(temp_list)):
                main_head = " ".join(temp_list)
                head_flag = True
                break
        
        # Check whether this sentence is a sub heading
        for head in sub_headings:
            flag, temp_list = compare(head, line)
            if(flag and len(temp_list)):
                sub_head = " ".join(temp_list)
                sub_flag = True
                break
        
        #print(i, head_flag, sub_flag)
        #print(line)
        
        # Adding current sentence data to a list
        text.append(line)
        pos1.append(i+1)
        labels.append(label)
        citations.append(citation)
        main_heading.append(main_head)
        sub_heading.append(sub_head)
        
        # If sentence is a main_heading
        if (head_flag == True):
            pos2.append(1)
            pos3.append(1)
        else:
            
            pos2.append(pos2[length]+1)
            # If sentence is a sub_heading
            if (sub_flag == True):
                pos3.append(1)
            else:
                pos3.append(pos3[length]+1)
        length += 1

../NCG_Dataset/training-data-master/paraphrase_generation/1
../NCG_Dataset/training-data-master/text_summarization/0
../NCG_Dataset/training-data-master/sentiment_analysis/51
../NCG_Dataset/training-data-master/sentiment_analysis/8
../NCG_Dataset/training-data-master/sentiment_analysis/5
../NCG_Dataset/training-data-master/sentiment_analysis/19
../NCG_Dataset/training-data-master/sentiment_analysis/35
../NCG_Dataset/training-data-master/semantic_role_labeling/3
../NCG_Dataset/training-data-master/topic_models/0
../NCG_Dataset/training-data-master/negation_scope_resolution/0
../NCG_Dataset/training-data-master/natural_language_inference/99
../NCG_Dataset/training-data-master/natural_language_inference/42
../NCG_Dataset/training-data-master/natural_language_inference/10
../NCG_Dataset/training-data-master/natural_language_inference/38
../NCG_Dataset/training-data-master/natural_language_inference/89
../NCG_Dataset/training-data-master/natural_language_inference/15
../NCG_Dataset/training

In [8]:
print(length)
print(count)

55200
19


In [ ]:
df = pd.DataFrame({"topic": topic_list, "paper_ID" : ID_list, "main_heading" : main_heading, 
                   "sub_heading" : sub_heading, "pos1" : pos1, "pos2" : pos2, "pos3" : pos3, 
                   "text" : text, "citation" : citations, "labels" : labels})

In [ ]:
def surrounding(df):
    text = df['text'].tolist()
    pos3 = df['pos3'].tolist()
    prev_text = []
    next_text = []
    for i, x in enumerate(pos3):
        if x > 2:
            prev_text.append(text[i-1])
        else:
            prev_text.append(" ")
        if x < 3:
            next_text.append(" ")
        else:
            next_text.append(text[i])
    next_text.pop(0)
    next_text.append(" ")
    df['prev_text'] = prev_text
    df['next_text'] = next_text
    return df

In [ ]:
df = surrounding(df)

In [ ]:
df.drop(columns = ["citation", "labels"], inplace = True)

In [ ]:
df

In [ ]:
df.sort_values(['topic', 'paper_ID', 'pos1', 'pos2', 'pos3'], ascending=[True, True, False, False, False], inplace=True)

p1, p2, p3 = 1,1,1
l1, l2, l3 = 1,1,1
ofs1 = []
ofs2 = []
ofs3 = []
for index, row in df.iterrows():
    if(row['pos1']>=p1):
        l1 = row['pos1']
    p1 = row['pos1']
    ofs1.append(p1/l1)
    if(row['pos2']>=p2):
        l2 = row['pos2']
    p2 = row['pos2']
    ofs2.append(p2/l2)
    if(row['pos3']>=p3):
        l3 = row['pos3']
    p3 = row['pos3']
    ofs3.append(p3/l3)

df['ofs1'] = ofs1
df['ofs2'] = ofs2
df['ofs3'] = ofs3

df.sort_values(['topic', 'paper_ID', 'pos1', 'pos2', 'pos3'], ascending=[True, True, True, True, True], inplace=True)

In [ ]:
df.to_csv("../Dataset_TaskA/1st_Jan/NCG_Dataset.csv", index = None)